In [16]:
import torch
import torchvision
from torchvision import transforms # import transforms from torchvision
import torch.nn as nn
import torch.nn.functional as fn
from google.colab import drive
drive.mount('/content/drive')

#we need to move the model to the GPU for faster processing
if torch.cuda.is_available():
  print("GPU is available!")
  device = torch.device("cuda")
else:
  print("GPU is not available.")
  device = torch.device("cpu")
#Using transforms we can prepare the dataset provided for training and testing
#The line below is responsible for converting the pixels in an image into a matrix containing numbers (tensor)
transforms  = transforms.Compose([transforms.ToTensor(),transforms.Resize((600,800)),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainers = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/ColabNotebooks/data/train', transform=transforms)
loadTrainer = torch.utils.data.DataLoader(trainers, batch_size=16, shuffle=True, num_workers=2)

dataTesters = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/ColabNotebooks/data/val', transform=transforms)#This line sets the classes that are intended to identify the dataset
testDataLoader = torch.utils.data.DataLoader(dataTesters, batch_size=16, shuffle=False, num_workers=2)
categories = ('Amphibia', 'Animalia', 'Arachnida', 'Aves','Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia')

#Now we can utilize a class to create the CNN with the necessary specifications
#Model adapted from GeeksforGeeks to meet specs
class CNN(nn.Module):
    def __init__(self): #must use self and cannot be another variable name
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.pool = nn.AdaptiveMaxPool2d(2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv5 = nn.Conv2d(512, 1024, 3, padding=1)
        self.fc1 = nn.Linear(1024 *2 *2, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, imageInput): # network to self
        imageInput = self.pool(fn.relu(self.conv1(imageInput)))
        imageInput = self.pool(fn.relu(self.conv2(imageInput)))
        imageInput = self.pool(fn.relu(self.conv3(imageInput)))
        imageInput = self.pool(fn.relu(self.conv4(imageInput)))
        imageInput = self.pool(fn.relu(self.conv5(imageInput)))

        imageInput = imageInput.view(-1, 1024 *2 *2)
        imageInput = fn.gelu(self.fc1(imageInput))
        imageInput = self.fc2(imageInput)
        return imageInput

net = CNN()
net = net.to(device)
#we need to create the loss function and the optimizer function
#we need to use cross entropy loss because this is a multi-class classification problem
loss = nn.CrossEntropyLoss()
#optimizer needs to update the weight of the models for training purposes
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

#Now we need to create the part of the code that actually trains the network
for iterations in range(2):
    run_loss = 0.0
    for i, images in enumerate(loadTrainer,0):
        inputs,labels = images
        print(inputs.shape, labels.shape)
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        #calculate the loss using the loss function, not the loss tensor
        current_loss = loss(outputs,labels)
        current_loss.backward()
        optimizer.step()
        run_loss += current_loss.item()
        if i %2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (iterations + 1, i + 1, run_loss / 2000))
            run_loss = 0.0

print('Training Done')

correct = 0
total = 0
with torch.no_grad():
    for data in testDataLoader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network: %d %%' % (100 * correct / total))
total_parameters = sum(p.numel() for p in net.parameters())
print(total_parameters)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU is available!
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.Size([16, 3, 600, 800]) torch.Size([16])
torch.S